In [1]:
import math
import os
from typing import Tuple
from tensorflow.python.keras.utils.generic_utils import Progbar
from tf_agents.environments import suite_gym, parallel_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.metrics import tf_metrics
from tf_agents.replay_buffers import tf_uniform_replay_buffer, episodic_replay_buffer
from tf_agents.trajectories.trajectory import Trajectory
from tf_agents.networks import actor_distribution_network
from tf_agents.drivers import dynamic_episode_driver, dynamic_step_driver
from tf_agents.policies import random_tf_policy
import tensorflow as tf
import numpy as np
from tf_agents.utils import common
from reinforcement_learning.sac_training import NumberOfSafetyViolations
from tf_agents.trajectories import time_step as ts, policy_step, trajectory
from reinforcement_learning import sac_training
from reinforcement_learning import labeling_functions
labeling_function = labeling_functions['Pendulum-v0']
from util.io.dataset_generator import map_rl_trajectory_to_vae_input
from util.io.dataset_generator import ErgodicMDPTransitionGenerator

In [2]:
py_env = suite_gym.load('Pendulum-v0')
py_env.reset()
tf_env = tf_py_environment.TFPyEnvironment(py_env)
tf_env.time_step_spec()


TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(3,), dtype=tf.float32, name='observation', minimum=array([-1., -1., -8.], dtype=float32), maximum=array([1., 1., 8.], dtype=float32)))

In [3]:
def display_safe_labeling(trajectory):
    label = labeling_functions['Pendulum-v0'](trajectory.observation)
    if tf.reduce_any(label):
        print(label)

In [3]:
replay_buffer_capacity = 1280
# specs
action_spec = tf_env.action_spec()
policy_step_spec = policy_step.PolicyStep(
    action=action_spec,
    state=(),
    info=())
trajectory_spec = trajectory.from_transition(tf_env.time_step_spec(),
                                             policy_step_spec,
                                             tf_env.time_step_spec())

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=trajectory_spec,
    batch_size=tf_env.batch_size,
    max_length=replay_buffer_capacity)

dataset_generator = lambda: replay_buffer.as_dataset(
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    num_steps=2
).map(
    map_func=lambda trajectory, _: map_rl_trajectory_to_vae_input(trajectory, labeling_function),
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    #  deterministic=False  # TF version >= 2.2.0
)

In [18]:
tf_env = tf_py_environment.TFPyEnvironment(py_env)

sac_policy_dir = '../saves/Pendulum-v0/policy/permissive_variance_policy-multiplier=15.0'
policy = tf.compat.v2.saved_model.load(sac_policy_dir)
dynamic_episode_driver.DynamicEpisodeDriver(tf_env, policy, num_episodes=5,
                                            observers=[
                                                # display_safe_labeling,
                                                lambda _: py_env.render(mode='human'),
                                                replay_buffer.add_batch
                                            ]).run()

(TimeStep(step_type=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>, reward=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>, discount=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>, observation=<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.80730134, -0.5901394 ,  0.9964641 ]], dtype=float32)>),
 ())

In [6]:
dataset = replay_buffer.as_dataset(
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    num_steps=2
)
iterator = iter(dataset)
trajectory, _ = next(iterator)

state = trajectory.observation[0, ...]
labels = tf.cast(labeling_function(trajectory.observation), tf.float32)
if tf.rank(labels) == 1:
    labels = tf.expand_dims(labels, axis=-1)
label = labels[0, ...]
action = trajectory.action[0, ...]
reward = trajectory.reward[0, ...]
if tf.rank(reward) == 1:
    reward = tf.expand_dims(reward, axis=-1)
next_state = trajectory.observation[1, ...]
next_label = labels[1, ...]

print("\nstate", state)
print('\nlabels', labels)
print('\nlabel', label)
print('\naction', action)
print('\nreward', reward)
print('\nnext_state', next_state)
print('\nnext_label', next_label)


state tf.Tensor([ 0.99578947 -0.09166978 -0.30986083], shape=(3,), dtype=float32)

labels tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)

label tf.Tensor([1. 1. 1.], shape=(3,), dtype=float32)

action tf.Tensor([1.4541345], shape=(1,), dtype=float32)

reward tf.Tensor(-0.020142874, shape=(), dtype=float32)

next_state tf.Tensor([ 0.99502176 -0.0996576  -0.16049299], shape=(3,), dtype=float32)

next_label tf.Tensor([1. 1. 1.], shape=(3,), dtype=float32)


In [8]:
generator = ErgodicMDPTransitionGenerator(labeling_function, replay_buffer)
dataset = replay_buffer.as_dataset(
    num_parallel_calls=4,
    num_steps=2
).map(
    map_func=generator,
    num_parallel_calls=4,
    #  deterministic=False  # TF version >= 2.2.0
).batch(batch_size=8, drop_remainder=True)
iterator = iter(dataset)
next(iterator)

(<tf.Tensor: shape=(8, 3), dtype=float32, numpy=
 array([[ 0.9996931 , -0.02477366,  0.13431005],
        [ 0.9979576 , -0.06387956,  0.29133275],
        [ 0.99604154, -0.08888876,  0.20481324],
        [ 0.99731207, -0.07327132,  0.02363477],
        [ 0.9993869 , -0.03501188, -0.33112368],
        [ 0.998584  , -0.05319791,  0.25009814],
        [ 0.9971614 , -0.07529349, -0.15001646],
        [ 0.9996998 , -0.02450128,  0.10047226]], dtype=float32)>,
 <tf.Tensor: shape=(8, 4), dtype=float32, numpy=
 array([[1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(8, 1), dtype=float32, numpy=
 array([[-0.64900994],
        [-0.08525274],
        [ 1.4327046 ],
        [ 1.3229486 ],
        [-0.02252978],
        [-0.3857172 ],
        [ 1.4918438 ],
        [-0.69871855]], dtype=float32)>,
 <tf.Tensor: sh

In [5]:
import variational_action_discretizer

vae_mdp = variational_action_discretizer.load(
    "../../saves/Pendulum-v0/models/vae_LS13_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.70-0.50_1e-06-2e-06/policy/action_discretizer/LA5_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.25-0.17_1e-06-2e-06_params=full_vae_optimization/step70000/eval_elbo-4.049"
    # "../../saves/Pendulum-v0/models/vae_LS13_MC1_ER20.0-decay=2e-05-min=-10_KLA0.0-growth=1e-06_TD0.70-0.50_1e-06-2e-06/policy/action_discretizer/LA5_MC1_ER20.0-decay=2e-05-min=-10_KLA0.0-growth=1e-06_TD0.25-0.17_1e-06-2e-06_params=full_vae_optimization/step270000/eval_elbo-1.704"
    # "../../saves/Pendulum-v0/models/vae_LS13_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.70-0.50_1e-06-2e-06/permissive_variance_policy-multiplier=10.0/action_discretizer/LA5_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.25-0.17_1e-06-2e-06_params=full_vae_optimization/step70000/eval_elbo-7.265"
)
print("VAE MDP loaded")

discrete_tf_env = vae_mdp.wrap_tf_environment(
    tf_env=tf_env,
    labeling_function=labeling_functions['Pendulum-v0'],
    deterministic_embedding_functions=False
)
discrete_tf_env.reset()

reward_metric = tf_metrics.AverageReturnMetric()

policy = vae_mdp.get_abstract_policy()

dynamic_episode_driver.DynamicEpisodeDriver(
    discrete_tf_env,
    policy,
    num_episodes=20,
    observers=[
        lambda _: py_env.render(mode='human'),
        reward_metric
    ]
).run()

print("avg rewards", reward_metric.result())

VAE MDP loaded
avg rewards tf.Tensor(-950.75665, shape=(), dtype=float32)
